In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Dropout,Activation
from tensorflow.python.keras import metrics
from sklearn.preprocessing import StandardScaler
from tensorflow.python import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

# 파일읽기

In [20]:
df=pd.read_excel('../optimal_data2/'+'Continous_2weeks_2day_1term.xlsx')
df.head()


,0,1,2,3,4,5,6,7,8
0,#AAGQKY,45.0,20.544193,72.0,0.0,0.0,0.0,148.0,0
1,#AAGQKY,45.0,20.544193,72.0,0.0,0.0,0.0,148.0,0
2,#AAGQKY,45.0,20.544193,72.0,0.0,0.0,0.0,148.0,0
3,#AAGQKY,45.0,20.544193,72.0,0.0,0.0,0.0,148.0,0
4,#AAGQKY,45.0,20.544193,72.0,0.0,0.0,0.0,148.0,0


In [21]:
df

,0,1,2,3,4,5,6,7,8
0,#AAGQKY,45.0,20.544193,72.0,0.0,0.0,0.0,148.0,0
1,#AAGQKY,45.0,20.544193,72.0,0.0,0.0,0.0,148.0,0
2,#AAGQKY,45.0,20.544193,72.0,0.0,0.0,0.0,148.0,0
3,#AAGQKY,45.0,20.544193,72.0,0.0,0.0,0.0,148.0,0
4,#AAGQKY,45.0,20.544193,72.0,0.0,0.0,0.0,148.0,0
...,...,...,...,...,...,...,...,...,...
26721,#ZZLSSL,68.0,29.048656,0.0,0.0,0.0,0.0,153.0,0
26722,#ZZLSSL,68.0,29.048656,0.0,0.0,0.0,0.0,153.0,0
26723,#ZZLSSL,68.0,29.048656,0.0,0.0,0.0,0.0,153.0,0
26724,#ZZLSSL,68.0,29.048656,0.0,0.0,0.0,0.0,153.0,0


# 데이터를 필요한 부분으로 X, Y 나눠 읽어오기

In [25]:
X=df.iloc[:,[1,3,4,5,6,7]]
y=df.iloc[:,-1]

scaler = StandardScaler()
X = scaler.fit_transform(X)
X=pd.DataFrame(X)

#결과 넣을 배열
Result=[[0 for j in range(4)] for i in range(10)]
Count=int(322/10)*83
pred_list=[]

In [26]:
#한 사람당 데이터 수
Count_1=int(83*0.1)
#한 사람당 데이터 수
Count_2=83

X_test=pd.DataFrame()
X_train=pd.DataFrame()
y_test=pd.DataFrame()
y_train=pd.DataFrame()
empty=pd.DataFrame()

#결과 넣을 배열
Result=[[0 for j in range(4)] for i in range(10)]

In [35]:
y_test

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
7258,2.0
7259,2.0
7260,2.0
7261,2.0


# 모델

## Deep Neural Network

## Input Feature : 6

## Hidden Layer : 128 - 256 - 128

## optimizer : adam, loss : categorical_crossentropy

In [29]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(6,)),
    keras.layers.Dense(128, activation=tf.nn.relu),
	keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
	keras.layers.Dense(3, activation=tf.nn.softmax),
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [32]:
for i in range(10):
    X_test=empty
    X_train=empty
    y_test=empty
    y_train=empty
    y_test_list=[]
    for j in range(322):
        X_temp_test=X.iloc[Count_2*j+Count_1*i:Count_2*j+Count_1*(i+1)]
        X_test=pd.concat([X_test,X_temp_test])
        X_temp_train=X.iloc[Count_2*j+Count_1:Count_2*(j+1)]
        X_train=pd.concat([X_train,X_temp_train])
        
        y_temp_test=y.iloc[Count_2*j+Count_1*i:Count_2*j+Count_1*(i+1)]
        y_test=pd.concat([y_test,y_temp_test])
        y_temp_train=y.iloc[Count_2*j+Count_1:Count_2*(j+1)]
        y_train=pd.concat([y_train,y_temp_train])
        
    
    print('SMOTE 적용 전 Train 레이블 값 분포: \n', y_train.value_counts())
    print('SMOTE 적용 전 Test 레이블 값 분포: \n', y_test.value_counts())
    
    # SMOTE 적용
    smote = SMOTE(random_state=0)
    X_train, y_train = smote.fit_resample(X_train,y_train)
    X_test,y_test = smote.fit_resample(X_test,y_test)
    print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_test.shape, y_test.shape)
    print('SMOTE 적용 후 Train 레이블 값 분포: \n', y_train.value_counts())
    print('SMOTE 적용 후 Test 레이블 값 분포: \n', y_test.value_counts())
    
    # 원핫인코딩
    ## 예시 : 1 , 2 -> (1,0) , (0,1)
    y_train=pd.get_dummies(y_train)
    y_test=pd.get_dummies(y_test)
    print(y_test)

SMOTE 적용 전 Train 레이블 값 분포: 
 0.0    22300
1.0     1194
2.0      656
dtype: int64
SMOTE 적용 전 Test 레이블 값 분포: 
 0.0    2290
1.0     161
2.0     125
dtype: int64
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (6870, 6) (6870, 1)
SMOTE 적용 후 Train 레이블 값 분포: 
 0.0    22300
1.0    22300
2.0    22300
dtype: int64
SMOTE 적용 후 Test 레이블 값 분포: 
 0.0    2290
1.0    2290
2.0    2290
dtype: int64
        0
0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
...   ...
6865  2.0
6866  2.0
6867  2.0
6868  2.0
6869  2.0

[6870 rows x 1 columns]
SMOTE 적용 전 Train 레이블 값 분포: 
 0.0    22300
1.0     1194
2.0      656
dtype: int64
SMOTE 적용 전 Test 레이블 값 분포: 
 0.0    2371
1.0     145
2.0      60
dtype: int64
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (7113, 6) (7113, 1)
SMOTE 적용 후 Train 레이블 값 분포: 
 0.0    22300
1.0    22300
2.0    22300
dtype: int64
SMOTE 적용 후 Test 레이블 값 분포: 
 0.0    2371
1.0    2371
2.0    2371
dtype: int64
        0
0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
...   ...
7108  2.0
7109  2.0
7110  2.0
7111  2.0
7112  2.0



In [33]:
y_test

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
7258,2.0
7259,2.0
7260,2.0
7261,2.0


In [31]:
    history=model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=6)
    predicted=model.predict(X_test)
    test_loss, test_acc = model.evaluate(X_test, y_test)

    accuracy=model.score(X_test,y_test)
    print("[{}]Accuracy : {}".format(i,accuracy))   
    #f1score
    f1 = f1_score(y_test,predicted, average='weighted')
    print("[{}]F1score : {}".format(i,f1))
    #precision/recall
    p_rlist=sk(y_test_list,predicted,average='weighted')
    print("[{}]Precision : {}".format(i,p_rlist[0]))
    print("[{}]Recall : {}".format(i,p_rlist[1]))
    print()
     #결과 배열에 넣기
    Result[i][0]=accuracy
    Result[i][1]=f1
    Result[i][2]=p_rlist[0]
    Result[i][3]=p_rlist[1]
    del accuracy
    del f1
    del p_rlist

SMOTE 적용 전 Train 레이블 값 분포: 
 0.0    22300
1.0     1194
2.0      656
dtype: int64
SMOTE 적용 전 Test 레이블 값 분포: 
 0.0    2290
1.0     161
2.0     125
dtype: int64
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (6870, 6) (6870, 1)
SMOTE 적용 후 Train 레이블 값 분포: 
 0.0    22300
1.0    22300
2.0    22300
dtype: int64
SMOTE 적용 후 Test 레이블 값 분포: 
 0.0    2290
1.0    2290
2.0    2290
dtype: int64
Epoch 1/100


ValueError: in user code:

    C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\tensorflow\python\keras\engine\training.py:855 train_function  *
        return step_function(self, iterator)
    C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\tensorflow\python\keras\engine\training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\jaemin\Anaconda3\envs\py38\lib\site-pack

# 예측 정확도를 계산하는 부분

In [ ]:
count=0
predicted_plot=[]
correct_plot=[]

count_array=[0 for j in range(3)]

predicted_result=[0 for j in range(len(test_y))]
for i in range(len(test_y)):
    maxIndex=tf.argmax(predicted[i])
    predicted_plot.append(maxIndex-1)
    #     print(type(maxIndex.numpy().item()))
    index=0
    if(y_origin[i]==0):
        index=0 
    elif y_origin[i]==1:
        index=1
    elif y_origin[i]==2:
        index=2
    predicted_result[i]=index-1
    if index==maxIndex.numpy().item():
        count=count+1
        count_array[index]=count_array[index]+1

# 예측 정확도

## 맞춘 개수 , 전체 개수 , 정확도

In [ ]:
print(count)
print(len(test_y))
print(count/len(test_y))

# 시각화하는 부분
## -1(감소): 100, 0(유지): 355, 1(증가):23

In [ ]:
label=['Decreasing','Maintain','Increasing']
values=[count_array[0]/8*100,count_array[1]/355*100,count_array[2]/23*100]

x = np.arange(3)

plt.bar(x, values)
plt.xticks(x, label)

plt.show()

In [ ]:
ratio = [count,len(test_y)-count]
labels = ['Correct', 'Wrong']

plt.pie(ratio, labels=labels, autopct='%.1f%%')
plt.show()